In [1]:
import matplotlib.pyplot as plt
import numpy as np

from datasets.mnist import MNIST

from models.cae_model import CAE
from models.conv_model import CNN

from train import train_ae, train_cnn

from cem import ContrastiveExplanationMethod

In [2]:
dataset = MNIST(batch_size=64)
# dataset = FashionMNIST()

# Training the classifier

In [ ]:
cnn = CNN()

train_cnn(cnn, dataset, iterations=2, lr=0.001, save_fn='mnist-cnn', device="cpu", load_path="")

In [ ]:
images, _ = dataset.get_batch()

output = cnn(images)

images = images.numpy()
output = output.detach().numpy()


# Training the autoencoder

This section trains the autoencoder which will be used as regularizer for the data space which the perturbations are found in.

In [ ]:
# Train or load autoencoder
cae = CAE()

train_ae(cae, dataset, iterations=2, save_fn="mnist-cae", device="cpu", load_path="")

In [3]:
# obtain one batch of test images
images, _ = dataset.get_batch()

#images_flatten = images.view(images.size(0), -1)
# get sample outputs
output = cae(images)
# prep images for display
images = images.numpy()

# output is resized into a batch of images
# output = output.view(batch_size, 1, 28, 28)
# use detach when it's an output that requires_grad
output = output.detach().numpy()

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=4, sharex=True, sharey=True, figsize=(25,4))

# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(np.squeeze(img), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

NameError: name 'cae' is not defined

# Contrastive Explanation Method

In [ ]:
CEM = ContrastiveExplanationMethod(cnn, cae, iterations=100, n_searches=4, kappa=10., gamma=1., beta=10., learning_rate=10)

In [ ]:
# obtain one sample
image, _ = dataset.get_sample()

plt.imshow(image.squeeze())

In [ ]:
# find pertinent negatives
CEM.fista(image, mode="PN")

In [ ]:
print(CEM.best_delta - image.view(28*28))

In [ ]:
plt.imshow(image.squeeze())

In [ ]:
# perturbation
plt.imshow(CEM.best_delta.view(28,28) - image.squeeze())

In [ ]:
# original image with perturbation
plt.imshow(CEM.best_delta.view(28,28))

In [ ]:
# classification after
np.argmax(cnn(CEM.best_delta.view(1,28,28)).detach())

In [ ]:
# classification before
np.argmax(cnn(image.squeeze(-1)).detach())

In [ ]:
print(CEM.best_loss)

In [ ]:
print(CEM.best_pert_loss)

In [4]:
print(images)

tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000]]],


        [[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000]]],


        [[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.500